In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stress-index-dataset/data_with_stress_index.csv


In [2]:
df_stress_index=pd.read_csv('../input/stress-index-dataset/data_with_stress_index.csv',usecols=lambda col:col not in ['Unnamed: 0'] )

In [3]:
df_stress_index.head()

,news_sentiment,broker_count_imbalance,volume_indicator,benchmark_price_difference,trade_count_imbalance,one_sided_trade_indicator,tranche_size_indicator,stress_index
0,0.393144,0.124362,0.677889,0.497776,-0.720698,-0.680076,0.724869,0.551515
1,0.772496,0.369376,-0.877238,0.139748,0.851864,-0.257519,0.586063,0.623805
2,-0.189353,0.190013,-0.485352,-0.364914,-0.183833,-0.235899,0.356616,0.305677
3,0.219090,-0.577930,0.876170,0.495462,0.182495,0.094340,0.106455,0.599768
4,0.038766,0.474529,0.345526,-0.153370,-0.020692,0.238055,0.432594,0.594587


In [4]:
%%capture
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q datasets
# !pip install evaluate
# !pip install -qqq trl==0.7.1
!pip install torch

In [5]:
import torch
import time
# import evaluate
import pandas as pd
import numpy as np
# from datasets import Dataset, load_dataset
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [6]:
import getpass

# Prompt for the Hugging Face token
hf_token = getpass.getpass("Enter your Hugging Face token: ")

import os
os.environ['HUGGINGFACE_TOKEN'] = hf_token

Enter your Hugging Face token:  ·····································


In [7]:
# model_id =  "NousResearch/Llama-2-7b-hf"
# model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "mistralai/Mistral-7B-v0.1"
model_id = "microsoft/phi-2"
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
# quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",token=hf_token)

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id,token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
[str({i:j for i,j in row.to_dict().items() if i not in ['stress_index','stress_index_bucket']}) for _, row in df_stress_index.head(4).iterrows()]

["{'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}",
 "{'news_sentiment': 0.7724956104208642, 'broker_count_imbalance': 0.3693763241525458, 'volume_indicator': -0.8772379830243947, 'benchmark_price_difference': 0.1397481154484952, 'trade_count_imbalance': 0.8518638934191982, 'one_sided_trade_indicator': -0.2575187071551993, 'tranche_size_indicator': 0.5860627411447076}",
 "{'news_sentiment': -0.1893525082726175, 'broker_count_imbalance': 0.1900129275888855, 'volume_indicator': -0.4853517528812924, 'benchmark_price_difference': -0.3649141815966704, 'trade_count_imbalance': -0.1838334364074593, 'one_sided_trade_indicator': -0.2358991365499189, 'tranche_size_indicator': 0.3566155592474426}",
 "{'news_sentiment': 0.21909032838

In [10]:
df_stress_index.loc[:,'stress_index_bucket']=pd.cut(df_stress_index['stress_index'],bins=10,labels=[str(np.round(x,1))+'-'+str(np.round(x+0.1,1)) for x in np.arange(0,1,0.1)])

In [11]:
df_stress_index.head()

,news_sentiment,broker_count_imbalance,volume_indicator,benchmark_price_difference,trade_count_imbalance,one_sided_trade_indicator,tranche_size_indicator,stress_index,stress_index_bucket
0,0.393144,0.124362,0.677889,0.497776,-0.720698,-0.680076,0.724869,0.551515,0.5-0.6
1,0.772496,0.369376,-0.877238,0.139748,0.851864,-0.257519,0.586063,0.623805,0.6-0.7
2,-0.189353,0.190013,-0.485352,-0.364914,-0.183833,-0.235899,0.356616,0.305677,0.3-0.4
3,0.219090,-0.577930,0.876170,0.495462,0.182495,0.094340,0.106455,0.599768,0.5-0.6
4,0.038766,0.474529,0.345526,-0.153370,-0.020692,0.238055,0.432594,0.594587,0.5-0.6


#### Trading Bot Prompt

In [12]:
import random
random.seed(42)

In [13]:
def shuffle_list(original_list):
    # Create a copy of the list
    shuffled_list = original_list.copy()

    # Shuffle the copy
    random.shuffle(shuffled_list)

    return shuffled_list

In [14]:
list_stress_index_bucket=[str(np.round(x,1))+'-'+str(np.round(x+0.1,1)) for x in np.arange(0,1,0.1)]
list_option_choices=[chr(x) for x in range(65,65+len(list_stress_index_bucket))]

In [15]:
print(list_option_choices)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']


**Prompt with 1 shot example (first row of the dataframe)**

In [16]:
instruction="""
###Instruction:
You are fixed income corporate bond trader. Use the feature values that I will provide and select the correct option for classifying the stress index (range of 0 to 1) into one of the defined buckets.
Feature descriptions:
- news_sentiment: Scale from -1 (negative) to +1 (positive) reflecting the sentiment in news about the bond issuer over the past 7 days.
- broker_count_imbalance: Difference in broker count buying or selling the same security in the last 2 days, ranging from -1 (more selling) to +1 (more buying).
- volume_indicator: Indicates if a bond is heavily traded in the last 2 days, with -1 for high selling volume and +1 for high buying volume.
- benchmark_price_difference: Compares a bond's quoted price to the benchmark, ranging from -1 (below benchmark) to +1 (above benchmark), indicating price stress.
- trade_count_imbalance: Difference in buy and sell trades of a security over 2 days, from -1 (more sells) to +1 (more buys).
- one_sided_trade_indicator: Imbalance in buy or sell trades over 7 days, with -1 for predominantly sell trades and +1 for buy trades.
- tranche_size_indicator: Assesses bond stress by the direction of trades and tranche size, ranging from 0 (large tranche) to 1 (small tranche).
###Question:
Given the feature value dict as:
{'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}
Return the correct option for classifying stress index from the following options based on above features. Do not generate anything else apart from the option.
"""

In [17]:
choice_prompt=''''''
for i,j in zip(list_option_choices,shuffle_list(list_stress_index_bucket)):
    choice_prompt+=f'({i}) {j}\n'

In [18]:
example_prompt=''''''
example_prompt+=instruction
example_prompt+=choice_prompt
example_prompt+='''
###Answer:
B
'''

In [19]:
print(example_prompt)


###Instruction:
You are fixed income corporate bond trader. Use the feature values that I will provide and select the correct option for classifying the stress index (range of 0 to 1) into one of the defined buckets.
Feature descriptions:
- news_sentiment: Scale from -1 (negative) to +1 (positive) reflecting the sentiment in news about the bond issuer over the past 7 days.
- broker_count_imbalance: Difference in broker count buying or selling the same security in the last 2 days, ranging from -1 (more selling) to +1 (more buying).
- volume_indicator: Indicates if a bond is heavily traded in the last 2 days, with -1 for high selling volume and +1 for high buying volume.
- benchmark_price_difference: Compares a bond's quoted price to the benchmark, ranging from -1 (below benchmark) to +1 (above benchmark), indicating price stress.
- trade_count_imbalance: Difference in buy and sell trades of a security over 2 days, from -1 (more sells) to +1 (more buys).
- one_sided_trade_indicator: Imb

In [22]:
final_prompt=''''''
final_prompt+=example_prompt
final_prompt+='''
###Question:
Given the feature value dict as:
{'news_sentiment': 0.7724956104208642, 'broker_count_imbalance': 0.3693763241525458, 'volume_indicator': -0.8772379830243947, 'benchmark_price_difference': 0.1397481154484952, 'trade_count_imbalance': 0.8518638934191982, 'one_sided_trade_indicator': -0.2575187071551993, 'tranche_size_indicator': 0.5860627411447076}
Return the correct option for classifying stress index from the following options based on above features. Do not generate anything else apart from the option.
'''
choice_prompt=''''''
for i,j in zip(list_option_choices,shuffle_list(list_stress_index_bucket)):
    choice_prompt+=f'({i}) {j}\n'

final_prompt+=choice_prompt
final_prompt+='''
###Answer:
'''

In [23]:
print(final_prompt)


###Instruction:
You are fixed income corporate bond trader. Use the feature values that I will provide and select the correct option for classifying the stress index (range of 0 to 1) into one of the defined buckets.
Feature descriptions:
- news_sentiment: Scale from -1 (negative) to +1 (positive) reflecting the sentiment in news about the bond issuer over the past 7 days.
- broker_count_imbalance: Difference in broker count buying or selling the same security in the last 2 days, ranging from -1 (more selling) to +1 (more buying).
- volume_indicator: Indicates if a bond is heavily traded in the last 2 days, with -1 for high selling volume and +1 for high buying volume.
- benchmark_price_difference: Compares a bond's quoted price to the benchmark, ranging from -1 (below benchmark) to +1 (above benchmark), indicating price stress.
- trade_count_imbalance: Difference in buy and sell trades of a security over 2 days, from -1 (more sells) to +1 (more buys).
- one_sided_trade_indicator: Imb

**Next Token Logits**

In [54]:
inputs = tokenizer(final_prompt, return_tensors='pt')

In [64]:
generated_ids=model.generate(
        inputs["input_ids"],
        temperature=0.2,
        max_new_tokens=1
)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1482: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') b

In [26]:
generated_ids.shape

torch.Size([1, 886])

In [65]:
output = tokenizer.decode(
    generated_ids[0],
        # stopping_criteria = [EosListStoppingCriteria()] ,
    skip_special_tokens=False
)

In [66]:
print(output)


###Instruction:
You are fixed income corporate bond trader. Use the feature values that I will provide and select the correct option for classifying the stress index (range of 0 to 1) into one of the defined buckets.
Feature descriptions:
- news_sentiment: Scale from -1 (negative) to +1 (positive) reflecting the sentiment in news about the bond issuer over the past 7 days.
- broker_count_imbalance: Difference in broker count buying or selling the same security in the last 2 days, ranging from -1 (more selling) to +1 (more buying).
- volume_indicator: Indicates if a bond is heavily traded in the last 2 days, with -1 for high selling volume and +1 for high buying volume.
- benchmark_price_difference: Compares a bond's quoted price to the benchmark, ranging from -1 (below benchmark) to +1 (above benchmark), indicating price stress.
- trade_count_imbalance: Difference in buy and sell trades of a security over 2 days, from -1 (more sells) to +1 (more buys).
- one_sided_trade_indicator: Imb

In [67]:
tokenizer.decode(generated_ids[0,-1])

'A'

In [29]:
import gc
del output
torch.cuda.empty_cache()

In [55]:
outputs=model(inputs["input_ids"])

In [74]:
output_logits = outputs.logits.detach().cpu()

In [75]:
next_token_logits = output_logits[:, -1]

In [76]:
tokenizer.convert_tokens_to_ids(['C','A'])

[34, 32]

In [77]:
next_token_logits[0,34]

tensor(19.2656)

In [78]:
next_token_logits[0,32]

tensor(19.5938)

In [68]:
def softmax(x):
    e_x=np.exp(x-np.max(x))
    return e_x/e_x.sum()


tokens_of_interest= list_option_choices
token_indices = tokenizer.convert_tokens_to_ids(tokens_of_interest)
indices_in_logits = {token: next_token_logits[0,token_idx].item() for token, token_idx in zip(tokens_of_interest, token_indices)}

label_to_softmax_dict=dict(zip(tokens_of_interest,softmax(np.array(list(indices_in_logits.values())))))

print(label_to_softmax_dict)

{'A': 0.2409698452200019, 'B': 0.10861361870672019, 'C': 0.17356406849601186, 'D': 0.12116724751538693, 'E': 0.14845721497652914, 'F': 0.08327695015793751, 'G': 0.055474408266582884, 'H': 0.027461823452697983, 'I': 0.016398203761495648, 'J': 0.024616619446635776}


In [35]:
import gc
# Clear memory

del inputs
del outputs
gc.collect()
torch.cuda.empty_cache()

In [69]:
def get_values_till_key(sorted_dict, key):
    values_before_key = []
    for k, v in sorted_dict.items():
        if k == key:
            values_before_key.append(v)
            break
        values_before_key.append(v)
    return values_before_key

#### LAC CONFORMAL SCORE

In [70]:
def lac(true_label, label_softmax_dict):
    lac_score = 1.0 - label_softmax_dict[true_label]
    return lac_score

In [71]:
##Calculating the LAC conformal score for the example
print(lac('H',label_to_softmax_dict))

0.972538176547302


#### APS CONFORMAL SCORE

In [72]:
def aps(true_label, label_softmax_dict):
    sorted_softmax_dict = dict(sorted(label_to_softmax_dict.items(), key=lambda item: item[1], reverse=True))
    high_labels = get_values_till_key(sorted_softmax_dict, true_label)
    aps_score = sum(high_labels)
    return aps_score

In [73]:
aps('H', label_to_softmax_dict)

0.9589851767918683